In [6]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib Inline

In [5]:
df = pd.read_csv('C:/Users/suhas/EDA_FULL_CASE_STUDY/model_trainer/cleaned.csv')

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OneHotEncoder,PowerTransformer,StandardScaler,MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV

import warnings


In [9]:
# top 5 rows
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,total score,average
0,female,group B,bachelor's degree,standard,none,72,72,74,218,72.666667
1,female,group C,some college,standard,completed,69,90,88,247,82.333333
2,female,group B,master's degree,standard,none,90,95,93,278,92.666667
3,male,group A,associate's degree,free/reduced,none,47,57,44,148,49.333333
4,male,group C,some college,standard,none,76,78,75,229,76.333333


In [10]:
# droping total score columns as was hsowing multicollinearity with average
df.drop(columns=['total score'],inplace =True)

In [12]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,average
0,female,group B,bachelor's degree,standard,none,72,72,74,72.666667
1,female,group C,some college,standard,completed,69,90,88,82.333333
2,female,group B,master's degree,standard,none,90,95,93,92.666667
3,male,group A,associate's degree,free/reduced,none,47,57,44,49.333333
4,male,group C,some college,standard,none,76,78,75,76.333333


In [15]:
# preparing X and Y vairbale
X = df.drop(columns = ['average'],axis =1)

In [16]:
y =df['average']

In [18]:
num_features = [feature for feature in X.columns if X[feature].dtype !='O']
num_features

['math score', 'reading score', 'writing score']

In [19]:
ohe_columns = ['gender','race/ethnicity	','parental level of education','lunch','test preparation course']

In [34]:
scale = ColumnTransformer(transformers=[('scale',StandardScaler(),slice(0,11))],remainder='passthrough')

In [35]:
scale

ColumnTransformer(remainder='passthrough',
                  transformers=[('scale', StandardScaler(),
                                 slice(0, 11, None))])

In [36]:
tnf = ColumnTransformer(transformers=[('ohe',OneHotEncoder(drop = 'first'),[0,1,2,3,4])],remainder='passthrough')

In [37]:
tnf

ColumnTransformer(remainder='passthrough',
                  transformers=[('ohe', OneHotEncoder(drop='first'),
                                 [0, 1, 2, 3, 4])])

In [38]:
# using linear regression
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state = 198)
pipe = Pipeline([
    ('tnf',tnf),
    ('scale',scale),
    ('model',LinearRegression(fit_intercept=True))
])
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
print('R2 score',r2_score(y_test,y_pred))
print('MEA',mean_absolute_error(y_test,y_pred))
print('MEA sqaured',mean_squared_error(y_test,y_pred))

R2 score 1.0
MEA 5.4474943074941016e-14
MEA sqaured 4.293002082503552e-27


In [42]:
#using Ridge Regression

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=198)

pipe = Pipeline([
    ('tnf',tnf),
    ('scale',scale),
    ('model',Ridge(fit_intercept=True))
])

pipe.fit(X_train,y_train)
#make predicton
y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)
#training set performamce 
model_train_r2score = r2_score(y_train,y_train_pred)
model_train_mea = mean_absolute_error(y_train,y_train_pred)

# Test set performance
model_test_r2score = r2_score(y_test,y_test_pred)
model_test_mea = mean_absolute_error(y_test,y_test_pred)

print('Model performance for Training set')
print('R2 {:.4f}'.format(model_train_r2score))
print('MEA {:.4f}'.format(model_train_mea))

print('Model performance for Testing set')
print('R2 {:.4f}'.format(model_test_r2score))
print('MEA {:.4f}'.format(model_test_mea))

Model performance for Training set
R2 1.0000
MEA 0.0000
Model performance for Testing set
R2 1.0000
MEA 0.0000


In [43]:
# using Lasso Regression
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.3, random_state=728)
pipe = Pipeline([
        ('tnf',tnf),
        ('scale',scale),
        ('model',Lasso(fit_intercept=True))
    ])
pipe.fit(X_train, y_train)

# Make predictions
y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)
# Training set performance
model_train_r2score = r2_score(y_train, y_train_pred) # Calculate r2score
model_train_mae = mean_absolute_error(y_train, y_train_pred) # Calculate MAE

# Test set performance
model_test_r2score = r2_score(y_test, y_test_pred) # Calculate r2score
model_test_mae = mean_absolute_error(y_test, y_test_pred) # Calculate MAE

  
print('Model performance for Training set')
print("R2: {:.4f}".format(model_train_r2score))
print("MAE: {:.4f}".format(model_train_mae))

print('Model performance for Test set')
print("R2: {:.4f}".format(model_test_r2score))
print("MAE : {:.4f}".format(model_test_mae))

Model performance for Training set
R2: 1.0000
MAE: 0.0600
Model performance for Test set
R2: 1.0000
MAE : 0.0601


In [47]:
# using Support vector Regrression
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.3, random_state=728)
pipe = Pipeline([
        ('tnf',tnf),
        ('scale',scale),
        ('model',SVR(C=1,gamma='auto',kernel='linear',max_iter=54000))
    ])
pipe.fit(X_train, y_train)

# Make predictions
y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)
# Training set performance
model_train_r2score = r2_score(y_train, y_train_pred) # Calculate r2score
model_train_mae = mean_absolute_error(y_train, y_train_pred) # Calculate MAE

# Test set performance
model_test_r2score = r2_score(y_test, y_test_pred) # Calculate r2score
model_test_mae = mean_absolute_error(y_test, y_test_pred) # Calculate MAE

  
print('Model performance for Training set')
print("R2: {:.4f}".format(model_train_r2score))
print("MAE: {:.4f}".format(model_train_mae))

print('Model performance for Test set')
print("R2: {:.4f}".format(model_test_r2score))
print("MAE : {:.4f}".format(model_test_mae))

Model performance for Training set
R2: 1.0000
MAE: 0.0404
Model performance for Test set
R2: 1.0000
MAE : 0.0428
